In [1]:
import os
import glob
import pandas as pd

# ---------- Paths ----------
BASE_DIR = r"C:\PG, IELTS, DOCS\research paper\poetry project"
RAW_DIR = os.path.join(BASE_DIR, "data_raw")
OUT_DIR = os.path.join(BASE_DIR, "data_processed")

os.makedirs(OUT_DIR, exist_ok=True)

OUTPUT_FILE = os.path.join(OUT_DIR, "poem_lookup_full.csv")


def normalize_poem_key(first_line: str) -> str:
    """Make a clean matching key from first line."""
    key = str(first_line).strip().lower()

    # Remove trailing punctuation and extra dots/spaces
    key = key.replace("…", " ")
    key = key.replace(". . .", " ")
    key = key.replace("...", " ")
    key = key.replace("—", " ")
    key = key.replace("-", " ")

    # Keep letters/numbers/spaces only
    key = "".join(ch if ch.isalnum() or ch.isspace() else " " for ch in key)
    key = " ".join(key.split())  # collapse spaces
    return key


def parse_block_file(xlsx_path: str) -> pd.DataFrame:
    """Parse one Block_X.xlsx where poems are line-by-line and poem ends when PoemType exists."""
    # Read first sheet
    df = pd.read_excel(xlsx_path)

    # Expecting columns like PoemName, PoemType (but we guard anyway)
    col_a = df.columns[0]
    col_b = df.columns[1] if len(df.columns) > 1 else None

    poems = []
    current_lines = []
    current_type = None

    block_name = os.path.splitext(os.path.basename(xlsx_path))[0]  # e.g., Block_7

    for _, row in df.iterrows():
        line_val = row.get(col_a, None)
        type_val = row.get(col_b, None) if col_b else None

        # Skip fully empty rows
        if pd.isna(line_val) and pd.isna(type_val):
            continue

        # If line exists, add to current poem lines
        if not pd.isna(line_val):
            line_text = str(line_val).strip()
            if line_text:
                current_lines.append(line_text)

        # If PoemType is present, this row marks END of poem
        if not pd.isna(type_val):
            current_type = str(type_val).strip()

            if current_lines:
                first_line = current_lines[0]
                full_text = " ".join(current_lines)

                poems.append({
                    "PoemKeyRaw": first_line,
                    "PoemKey": normalize_poem_key(first_line),
                    "text": full_text,
                    "PoemType": current_type,
                    "Block": block_name
                })

            # Reset for next poem
            current_lines = []
            current_type = None

    return pd.DataFrame(poems)


def main():
    block_files = sorted(glob.glob(os.path.join(RAW_DIR, "Block_*.xlsx")))

    if not block_files:
        print("No Block_*.xlsx files found in:", RAW_DIR)
        return

    print("Found block files:")
    for f in block_files:
        print(" -", os.path.basename(f))

    all_lookup = []
    for bf in block_files:
        part = parse_block_file(bf)
        print(f"{os.path.basename(bf)} -> {len(part)} poems")
        all_lookup.append(part)

    lookup_df = pd.concat(all_lookup, ignore_index=True)

    # Drop duplicates by normalized key if any (keep first)
    lookup_df = lookup_df.drop_duplicates(subset=["PoemKey"]).reset_index(drop=True)

    lookup_df.to_csv(OUTPUT_FILE, index=False)

    print("\nDone!")
    print("Saved poem lookup to:", OUTPUT_FILE)
    print("Total poems:", len(lookup_df))
    print("\nPreview:")
    print(lookup_df.head())


if __name__ == "__main__":
    main()

Found block files:
 - Block_1.xlsx
 - Block_2.xlsx
 - Block_3.xlsx
 - Block_4.xlsx
 - Block_5.xlsx
 - Block_6.xlsx
 - Block_7.xlsx
Block_1.xlsx -> 30 poems
Block_2.xlsx -> 30 poems
Block_3.xlsx -> 30 poems
Block_4.xlsx -> 30 poems
Block_5.xlsx -> 30 poems
Block_6.xlsx -> 30 poems
Block_7.xlsx -> 30 poems

Done!
Saved poem lookup to: C:\PG, IELTS, DOCS\research paper\poetry project\data_processed\poem_lookup_full.csv
Total poems: 210

Preview:
                                          PoemKeyRaw  \
0  harvest festival\njars of fig jam\nfull of gal...   
1  nights drawing in—\nwondering how Dad is\nin h...   
2  a year at most . . . \nwe pretend to watch \nt...   
3  Mars landing\ncardboard softens\nthe subway grate   
4   the heft\nof a cast-iron skillet\nautumn deepens   

                                             PoemKey  \
0  harvest festival jars of fig jam full of galaxies   
1  nights drawing in wondering how dad is in his ...   
2  a year at most we pretend to watch the hummin